# 🇲🇬 Voambolana Malagasy — NLP Project Explorer

Ce notebook est le centre de contrôle pour l'entraînement de l'IA Malgache.

---

## 🛠️ 1. Configuration (Drive + Git)
**Exécutez cette cellule à chaque démarrage** pour monter votre Drive et récupérer les derniers scripts.

In [ ]:
from google.colab import drive
import os

print("🔗 Connexion à Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Drive monté avec succès !")
except Exception as e:
    print("❌ Erreur lors du montage du Drive.")

REPO_NAME = "nlp_malagasy"
BASE_PATH = "/content/drive/MyDrive/" + REPO_NAME

if not os.path.exists(BASE_PATH):
    %cd /content/drive/MyDrive
    !git clone https://github.com/franck504/nlp_malagasy.git

%cd $BASE_PATH
!git pull origin main

print(f"\n📍 Position actuelle : {os.getcwd()}")

## 🏗️ PARTIE 1 : COLLECTE & SCRAPING (PHASE 1 - 4)
Collecte des paroles de chansons depuis tononkira.serasera.org.

In [ ]:
!pip install requests beautifulsoup4
%cd $BASE_PATH/tononkira_rehetra

print("--- PHASE 1 : DISCOVERY ---")
!python 01_discover_artists.py --delay 1.0

print("\n--- PHASE 2 : SCRAPING (TURBO) ---")
!python 02_scrape_lyrics.py --delay 0.5 --artist-workers 4 --song-workers 10

print("\n--- PHASE 3 : STATISTIQUES ---")
!python 03_stats.py

print("\n--- PHASE 4 : FUSION DU CORPUS BRUT ---")
!python 04_merge_corpus.py

## 🧹 PARTIE 2 : ENRICHISSEMENT & CONSOLIDATION (PHASE 4.5)
Nettoyage des langues étrangères et ajout de Wikipedia Malagasy + Bible.

In [ ]:
%cd $BASE_PATH/tononkira_rehetra

print("--- PHASE 4.5a : PURIFICATION (Nettoyage de langue) ---")
!python 05_clean_corpus.py --input malagasy_lyrics_corpus.txt --output malagasy_lyrics_cleaned.txt

print("\n--- PHASE 4.5b : WIKIPEDIA MALAGASY (Extraction) ---")
!python 10_download_wikipedia.py
!python 11_extract_wiki.py

print("\n--- PHASE 4.5c : CONSOLIDATION FINALE (Corpus de 812k blocs) ---")
!python 06_consolidate_corpus.py --lyrics malagasy_lyrics_cleaned.txt --bible ../from_bible_json --wiki ../malagasy_wikipedia_raw.txt --output malagasy_corpus_v2_final.txt

## 🧠 PARTIE 3 : FONDATIONS NLP & IA (PHASE 5)
Entraînement du cerveau de l'IA (Vecteurs et Morphologie).

In [ ]:
!pip install tokenizers gensim
%cd $BASE_PATH/tononkira_rehetra

print("--- PHASE 5a : TOKENIZATION (Morphologie Malgache) ---")
!python 07_train_tokenizer.py --corpus malagasy_corpus_v2_final.txt --output tokenizer_mg

print("\n--- PHASE 5b : NORMALISATION (Nettoyage Embeddings) ---")
!python 09_normalize_corpus.py --input malagasy_corpus_v2_final.txt --output malagasy_corpus_normalized.txt

print("\n--- PHASE 5c : EMBEDDINGS (Vecteurs de sens FastText) ---")
!python 08_train_embeddings.py --corpus malagasy_corpus_normalized.txt --output embeddings_mg

## 🚀 PARTIE 4 : APPLICATIONS & DÉMO (PHASE 6)
Utilisation concrète du moteur de recherche sémantique.

### 🏗️ 1. Indexation Sémantique des 14 000 chansons

> [!IMPORTANT]
> **BOUST TURBO** : Exécutez le script 13 d'abord pour contourner la lenteur de Google Drive.

In [ ]:
%cd $BASE_PATH/tononkira_rehetra

print("--- PHASE 6a : REGROUPEMENT TURBO (Bypass Drive Latency) ---")
!python 13_bundle_songs.py

print("\n--- PHASE 6b : INDEXATION SÉMANTIQUE ---")
!python 12_test_app.py --index

### 🔍 2. Tableau de Bord Interactif (Phase 6c)
C'est ici que vous parlez avec l'intelligence de votre IA.

In [ ]:
%cd $BASE_PATH/tononkira_rehetra

RECHERCHE_SENS = "Hira momba ny fitiavana tanindrazana" # @param {type:"string"}
CORRECTION_MOT = "fityavana"                           # @param {type:"string"}
EXPLORER_CONCEPT = "Andriamanitra"                      # @param {type:"string"}
STYLE_A_DETECTER = "Tamin'ny voalohany Andriamanitra"   # @param {type:"string"}

print(f"--- 🔍 RECHERCHE SÉMANTIQUE : {RECHERCHE_SENS} ---")
!python 12_test_app.py --query "{RECHERCHE_SENS}"

print(f"\n--- 📝 CORRECTION : {CORRECTION_MOT} ---")
!python 12_test_app.py --spell "{CORRECTION_MOT}"

print(f"\n--- 🧠 ANALYSE DE CONCEPT : {EXPLORER_CONCEPT} ---")
!python 12_test_app.py --explore "{EXPLORER_CONCEPT}"

print(f"\n--- 🎭 DÉTECTION DE STYLE ---")
!python 12_test_app.py --style "{STYLE_A_DETECTER}"

print(f"\n--- ⚖️ ANALOGIE : Mpanjaka - Lehilahy + Vehivavy ---")
!python 12_test_app.py --analogy lehilahy mpanjaka vehivavy